In [323]:
import re
from notebooks.sentiment.util import build_pipeline, print_eval, preprocess_tweets
from sentiment.new_data import InterTASSAugmented
from sentiment.tass import InterTASSReader
from tqdm import tqdm
import autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [345]:
# reader = InterTASSAugmented(ratio=0.9)  # Class to use augmented data
# X_train, y_train = reader.Xy()
train = "intertass-ES-train-tagged.xml"
train = InterTASSReader(train)
X_train, y_train = list(train.X()), list(train.y())


corpus = "intertass-ES-development-tagged.xml"
dev = InterTASSReader(corpus)
X_dev, y_dev = list(dev.X()), list(dev.y())

pipeline = build_pipeline()
pipeline.fit(X_train, y_train)

/home/gosen/.local/share/virtualenvs/PLN-2019-N1iczKTA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/gosen/.local/share/virtualenvs/PLN-2019-N1iczKTA/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('feats', FeatureUnion(n_jobs=None,
       transformer_list=[('vect', TfidfVectorizer(analyzer='char_wb', binary=True, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=5,
        ngr...penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])

In [325]:
print_eval(pipeline, X_dev, y_dev)

accuracy	0.56

              precision    recall  f1-score   support

           n       0.58      0.77      0.66       219
           p       0.20      0.07      0.11        69
        none       0.24      0.10      0.14        62
         neu       0.61      0.65      0.63       156

   micro avg       0.56      0.56      0.56       506
   macro avg       0.41      0.40      0.38       506
weighted avg       0.50      0.56      0.51       506

[[168  11   9  31]
 [ 40   5   4  20]
 [ 37   5   6  14]
 [ 44   4   6 102]]


In [326]:
y_pred = pipeline.predict(X_dev)

In [327]:

import pandas as pd
# pd.options.display.max_colwidth = 0

errors = []
for x, y1, y2 in zip(X_dev, y_dev, y_pred):
    if y1 != y2:
        errors.append({
            'item': x,
            'true': y1,
            'pred': y2})

errdf = pd.DataFrame(errors)
errdf['len'] = errdf['item'].apply(lambda x: len(x))

In [328]:
labels = {"NONE": 2, "N":0, "P": 3, "NEU": 1}

In [329]:
y_prob = pipeline.predict_proba(X_dev)

In [330]:
import pandas as pd
import numpy as np
# pd.options.display.max_colwidth = 0

errors = []
for i, (x, y1, y2, y2p) in enumerate(zip(X_dev, y_dev, y_pred, y_prob)):
    if y1 != y2:
        assert np.argmax(y2p) == labels[y2]
        label_pred, label_y = labels[y2], labels[y1]
        diff = y2p[label_pred] - y2p[label_y]
        errors.append({
            'index': i,
            'item': x,
            'true': y1,
            'pred': y2,
            'diff': diff})

errdf = pd.DataFrame(errors)
errdf.sort_values('diff', inplace=True, ascending=False)

In [331]:
errdf[:10] # 10 tweets peor clasificados

,diff,index,item,pred,true
134,0.443618,277,Mi madre me deja ponerme rubia pero no el pelo morado,N,NEU
207,0.441336,454,@Pattvh_ no porque no los tienes activados a todos,N,NONE
14,0.429036,28,y lo peor de todo es que funcionaba maldita Jaco como te quiero,N,NEU
177,0.418879,374,@Idolsftdjom @_jesus_OM @DanieloviedoM obviamente tia JAJAJAJAJA q soy la mayor osqqqqq pero venir ya ofu,N,NONE
40,0.411710,100,"@gilthoniel_1987 Si estoy trabajando 😭, con vistas eso sí. Buenos días!!",P,NEU
141,0.403224,291,"@AnaSJuarez @OfficialMauiJim ¡Hola Ana! Te hemos contestado por mensaje privado, donde no hay limitación de caracteres ¡Gracias!",P,NONE
100,0.393545,209,@ArthasSama @Holic_meri es por no destrozar tus posibilidades con la gente guapa,P,NEU
107,0.384047,223,"Cuando no puedo dormir, escribo todo lo que preocupa en una libreta que alguien me regaló y es como un somnífero instantáneo",N,P
153,0.378016,320,A mí nunca me podrán hacer una broma porque no cojo llamadas y menos cuando son ocultas,N,NONE
32,0.359245,74,"@TheFuzzCanyon gracias por etiquetarme, pero no suelo hacer estas cosas",P,NEU


In [332]:
y_prob[291], y_pred[291], y_dev[291]

(array([0.17939236, 0.09067096, 0.16335655, 0.56658013]), 'P', 'NONE')

In [333]:
tweet = errdf.loc[141]["item"]
tweet

'@AnaSJuarez @OfficialMauiJim ¡Hola Ana! Te hemos contestado por mensaje privado, donde no hay limitación de caracteres  ¡Gracias!'

### Sabemos por el ejercicio anterior lo siguiente:
    - El peso positivo de los signos de exclamación es alto 

In [334]:
vect = pipeline.named_steps['feats']
clf = pipeline.named_steps['clf']
coef = clf.coef_

pos_coef = coef[labels["P"]]
none_coef = coef[labels["NONE"]]

In [335]:
import import_ipynb
import tabulate
from sentiment.analysis import prettify_names, print_feature_weights_for_item

features = vect.get_feature_names()
features = list(map(prettify_names, features))

features_df = pd.DataFrame({'name': features, 'pos_coefs': pos_coef.ravel(),
                            'none_coefs': none_coef.ravel()})
features_df.sort_values('pos_coefs', inplace=True, ascending=False)

def search_features(names, features_df):
    res = []
    for name in names:
        res.append(features_df[features_df["name"] == name])
    data = pd.DataFrame(pd.concat(res))
    return data

In [336]:
features_df[:10]

,name,pos_coefs,none_coefs
6681,!,1.115518,-0.172544
1482,!,0.947761,-0.119474
1483,!,0.917786,-0.109306
6228,uen,0.687026,-0.295466
24866,guapa,0.668180,-0.150822
3497,gra,0.592746,-0.433979
7077,#,0.591116,-0.124995
692,g,0.588155,-0.344366
5323,rac,0.546915,-0.018532
5324,raci,0.544964,-0.197824


In [337]:
tweet

'@AnaSJuarez @OfficialMauiJim ¡Hola Ana! Te hemos contestado por mensaje privado, donde no hay limitación de caracteres  ¡Gracias!'

In [338]:
search_features(tweet.split(" "), features_df)

,name,pos_coefs,none_coefs
37903,¡Hola,0.011822,0.090339
15676,Te,0.033012,-0.054424
1800,Te,-0.109241,0.104625
5184,por,-0.000409,-0.274191
28268,mensaje,0.133110,-0.051975
4612,no,-0.175268,-0.477257
3589,hay,0.128032,-0.233209
2714,de,-0.035710,0.006082
19255,caracteres,-0.035744,-0.014420


Como podemos ver el peso de los signos de exclamacion es muy influeyente en la clase positiva,
e incluso parece desfavorecer a la clase "none".
Probemos sacando los signos de exclamacion a la oracion.


In [339]:
pipeline.predict([re.sub(r"!|¡", "", tweet)])[0]

'P'

El modelo sigue prediciendo la oración como positiva, veamos los pesos ahora:

In [340]:
table = search_features(re.sub(r"!|¡", "", tweet).split(" "), features_df)
table

,name,pos_coefs,none_coefs
1666,Hola,0.019103,-0.024785
12717,Hola,0.016358,-0.117871
15676,Te,0.033012,-0.054424
1800,Te,-0.109241,0.104625
5184,por,-0.000409,-0.274191
28268,mensaje,0.133110,-0.051975
4612,no,-0.175268,-0.477257
3589,hay,0.128032,-0.233209
2714,de,-0.035710,0.006082
19255,caracteres,-0.035744,-0.014420


Vemos que por mas que los signos de exclamacion sumaban peso para la clase positiva,
aún hay palabras como "hola" que restan en "none" cuando no deberia ser así, o por ejemplo
la palabra "no" tiene mayor peso negativo en none que en los positivos, cosa que tambien parece
incorrecto

In [341]:
none_feat = pd.DataFrame({'name': features,
                            'none_coefs': none_coef.ravel()})
none_feat.sort_values('none_coefs', inplace=True, ascending=False)
none_feat[:5]

,name,none_coefs
10573,?,1.142083
1548,?,1.024874
1549,?,0.984363
26748,juntos,0.926088
16752,abstracto,0.910561


Probemos modificar un poco la oracion, para que nuestro modelo la interprete como debería ser

In [342]:
new_tweet = '@AnaSJuarez @OfficialMauiJim Hola Ana juntos te hemos contestado por \
            mensaje privado, donde no hay limitación de caracteres ? Gracias ?'

In [343]:
pred = pipeline.predict_proba([new_tweet])

In [344]:
from collections import OrderedDict
from operator import itemgetter

sorted_dict = sorted(labels.items(), key=itemgetter(1))
sorted_dict, pred

([('N', 0), ('NEU', 1), ('NONE', 2), ('P', 3)],
 array([[0.18464505, 0.05735553, 0.32921609, 0.42878333]]))

El modelo sigue prediciendo como positiva la oración, tendriamos que deformarla demasiado
agregando muchos signos de pregunta o palabras que favorezcan a "NONE" para que el modelo
la prediga bien.

En conclusión, esto puede deberse al desbalanceo que hay en el dataset de tweets, es decir que para
que el entrenamiento fuera mas certero habria que tener aproximadamente la misma cantidad de
tweets de cada clase, y en nuestro dataset, la mayoria de los tweets tienen clase positiva o negativa. 